In [1]:
import random as rd
import numpy as np
import h5py
import klepto
import shelve
import pickle
import scipy.io as sio
from scipy.sparse import *

c:\users\zzhno\anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Input Prediction Model

In [2]:
import shelve
import pickle
prediction_model = {}
tree_num = 3
dataset = '1m'

for i in range(tree_num):
    print(i + 1)
    with shelve.open("test_" + dataset + "/" + str(i + 1) + "/P_test", protocol=pickle.HIGHEST_PROTOCOL) as d:
        P_test = d['content']
    with shelve.open("test_" + dataset + "/" + str(i + 1) + "/rated_matrix", protocol=pickle.HIGHEST_PROTOCOL) as d:
        rated_matrix = d['content']
    prediction_model[i] = {'P_test': P_test, 'rated_matrix': rated_matrix}

1
2
3


## Input Test item info

In [3]:
feature=h5py.File('movielens/' + dataset + '/test_list.mat')  
test_list = feature['test_list'][:]
test_list = list(map(int, test_list.T[0]))
test_list = [i-1 for i in test_list] 
feature=h5py.File('movielens/' + dataset + '/train_list.mat')  
train_list = feature['train_list'][:]
train_list = list(map(int, train_list.T[0]))
train_list = [i-1 for i in train_list] 

In [4]:
rating_matrix = load_npz('movielens/sparse_matrix_ml-' + dataset + '_selected.npz').tocsc()
rating_matrix_train = rating_matrix[:, train_list]
rating_matrix_test  = rating_matrix[:, test_list]
print("file load DONE")
rating_matrix_test = rating_matrix_test.toarray()

file load DONE


## RMSE of model, RMSE of different rating 

In [5]:
tree_res = {}
for i in range(tree_num):
    tree_res[i] = []
for itemid in range(rating_matrix_test.shape[1]):
    model_ind = rd.randint(0, tree_num-1)
    tree_res[model_ind].append(itemid)

In [6]:
with shelve.open("tree_res", protocol=pickle.HIGHEST_PROTOCOL) as d:
    d['content'] = tree_res

In [7]:
with shelve.open("tree_res", protocol=pickle.HIGHEST_PROTOCOL) as d:
    tree_res = d['content']

In [7]:
P_test = np.zeros(rating_matrix_test.shape)
rated_matrix = np.zeros(rating_matrix_test.shape)

for i in range(tree_num):
    print(i)
    P_test[:, tree_res[i]] = (prediction_model[i]['P_test'].tocsc())[:, tree_res[i]].toarray()
    rated_matrix[:, tree_res[i]] = (prediction_model[i]['rated_matrix'])[:, tree_res[i]]

P_test *= rated_matrix
rating_matrix_test_unqueried = rating_matrix_test * rated_matrix 
del prediction_model

dif_pow = P_test - rating_matrix_test_unqueried
del P_test
dif_pow =  dif_pow * dif_pow

0
1
2


In [8]:
RMSE = (dif_pow.sum() / (rating_matrix_test_unqueried!=0).sum() )**0.5
RMSE

0.9229297143797129

In [9]:
control = rating_matrix_test_unqueried > 4.5
RMSE_1 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_1

1.1817059379238501

In [10]:
control = ((rating_matrix_test_unqueried > 3.5) * 1) * ((rating_matrix_test_unqueried <= 4.5) * 1)
RMSE_2 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_2

0.6162024496338405

In [11]:
control = ((rating_matrix_test_unqueried > 2.5) * 1) * ((rating_matrix_test_unqueried <= 3.5) * 1)
RMSE_3 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_3

0.6050199758146687

In [12]:
control = ((rating_matrix_test_unqueried > 1.5) * 1) * ((rating_matrix_test_unqueried <= 2.5) * 1)
RMSE_4 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_4

1.1264747702208164

In [13]:
control = ((rating_matrix_test_unqueried != 0) * 1) * ((rating_matrix_test_unqueried <= 1.5) * 1)
RMSE_5 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_5

1.7783487383336931

- 20m

In [9]:
RMSE = (dif_pow.sum() / (rating_matrix_test_unqueried!=0).sum() )**0.5
RMSE

0.8796851258267778

In [11]:
control = rating_matrix_test_unqueried > 4.5
RMSE_1 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_1

1.1945656569597722

In [12]:
control = ((rating_matrix_test_unqueried > 3.5) * 1) * ((rating_matrix_test_unqueried <= 4.5) * 1)
RMSE_2 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_2

0.6714199640848323

In [13]:
control = ((rating_matrix_test_unqueried > 2.5) * 1) * ((rating_matrix_test_unqueried <= 3.5) * 1)
RMSE_3 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_3

0.5329258656027343

In [14]:
control = ((rating_matrix_test_unqueried > 1.5) * 1) * ((rating_matrix_test_unqueried <= 2.5) * 1)
RMSE_4 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_4

0.9729791924078471

In [12]:
control = ((rating_matrix_test_unqueried != 0) * 1) * ((rating_matrix_test_unqueried <= 1.5) * 1)
RMSE_5 = ((dif_pow * control).sum() / control.sum() )**0.5
RMSE_5

1.8282835089670466

## Multiple Trees

In [9]:
selectNum = 9

#new items for each tree
tree_res = {}

P_test = np.zeros(rating_matrix_test.shape)
rated_matrix = np.zeros(rating_matrix_test.shape)
rated_matrix = rated_matrix == 0

for i in range(tree_num):
    tree_res[i] = []
for itemid in range(rating_matrix_test.shape[1]):
    rand_list = rd.sample(list(range(10)), selectNum) 
    for j in rand_list:
        tree_res[j].append(itemid)
        rated_matrix[:, itemid] = rated_matrix[:, itemid] * prediction_model[j]['rated_matrix'][:, itemid]

for i in range(tree_num):
    P_test[:, tree_res[i]] += (prediction_model[i]['P_test'].tocsc())[:, tree_res[i]].toarray()

P_test *= rated_matrix
P_test /= selectNum

rating_matrix_test_unqueried = rating_matrix_test * rated_matrix 
dif = P_test - rating_matrix_test_unqueried
del P_test
del rated_matrix
RMSE = ((dif ** 2).sum() / (rating_matrix_test_unqueried!=0).sum() )**0.5

In [9]:
RMSE #1

0.8797330233013486

In [11]:
RMSE #2

0.8697207456908191

In [13]:
RMSE #3

0.8661427117047424

In [15]:
RMSE #4

0.8648910628647799

In [27]:
RMSE #5

0.8642835845593122

In [29]:
RMSE #6

0.8642830145940552

In [31]:
RMSE #7

0.8652061242546697

In [6]:
RMSE #8

0.86551417945201

In [6]:
RMSE #9

0.8663719295534278

In [10]:
RMSE #10

0.8660629056265818